<a href="https://colab.research.google.com/github/frannfuri/hello-world/blob/master/Identif_voz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Montar directorio google drive en Colaboratory
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/Colab Notebooks/Proyecto_Aprendizaje

/content/drive/My Drive/Colab Notebooks/Proyecto_Aprendizaje


In [4]:
!ls

Identif_voz.ipynb


# Extraccion de features

In [0]:
import os
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)

def get_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img) # transforma imagen a un numpy array
    x = np.expand_dims(x, axis=0) # agrega un eje en la posicion axis=0
    x = preprocess_input(x)
    flatten = model.predict(x)
    return list(flatten[0])

X = []
y = []
num_classes = 20


audio_espectograms = []
ids = []

for img_filename in os.listdir('/content/drive/My Drive/Colab Notebooks/Proyecto_Aprendizaje/train'):
    if img_filename.endswith(".jpg"):
        ids.append(img_filename.split('-')[0])
        path = '/content/drive/My Drive/Colab Notebooks/Proyecto_Aprendizaje/train' + img_filename
        audio_espectograms.append(path)
        
for i in len(audio_espectograms):
  X.append(get_features(audio_espectograms[i]))
  y.append(ids[i])
        

################################# Hasta aqui nomas pescar #########################  

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(X_train, y_train)

predicted = clf.predict(X_test)

# get the accuracy
print (accuracy_score(y_test, predicted))